In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import Neccessary libraries
import numpy as np 
import pandas as pd 

# Import Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Import Model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from imblearn.pipeline import Pipeline as imbPipeline

# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

#%pip install vecstack

In [2]:
df = pd.read_csv("E:/Lakni UOC/4th Year/1st sem/ST 4052 - Statistical Learning II/2.grp project 1/codes_diabetes/diabetes_prediction_dataset.csv")

In [3]:
df_copy = df.copy()

In [4]:
df_copy.drop_duplicates(inplace=True)
print(f'without duplicates:{df_copy.shape}')

# Remove Unneccessary value [0.02%]
df2 = df_copy[df_copy['gender'] != 'Other']

# Define a function to map the existing categories to new ones
def recategorize_smoking(smoking_status):
    if smoking_status == 'never':
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'
    else:
        return 'no info'
    
# Apply the function to the 'smoking_history' column
df2['smoking_history'] = df2['smoking_history'].apply(recategorize_smoking)

#encoding
gender = {'Female':0,'Male':1}
df2['gender'] =df2['gender'].replace(gender)

def perform_one_hot_encoding(df2, column_name):
    # Perform one-hot encoding on the specified column
    dummies = pd.get_dummies(df2[column_name], prefix=column_name)

    # Drop the original column and append the new dummy columns to the dataframe
    df2 = pd.concat([df2.drop(column_name, axis=1), dummies], axis=1)

    return df2

# Perform one-hot encoding on the smoking history variable
data = perform_one_hot_encoding(df2, 'smoking_history')
data.head()

without duplicates:(96146, 9)


,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_current,smoking_history_no info,smoking_history_non-smoker,smoking_history_past_smoker
0,0,80.00,0,1,25.19,6.60,140,0,0,0,1,0
1,0,54.00,0,0,27.32,6.60,80,0,0,1,0,0
2,1,28.00,0,0,27.32,5.70,158,0,0,0,1,0
3,0,36.00,0,0,23.45,5.00,155,0,1,0,0,0
4,1,76.00,1,1,20.14,4.80,155,0,1,0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96128 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       96128 non-null  int64  
 1   age                          96128 non-null  float64
 2   hypertension                 96128 non-null  int64  
 3   heart_disease                96128 non-null  int64  
 4   bmi                          96128 non-null  float64
 5   HbA1c_level                  96128 non-null  float64
 6   blood_glucose_level          96128 non-null  int64  
 7   diabetes                     96128 non-null  int64  
 8   smoking_history_current      96128 non-null  uint8  
 9   smoking_history_no info      96128 non-null  uint8  
 10  smoking_history_non-smoker   96128 non-null  uint8  
 11  smoking_history_past_smoker  96128 non-null  uint8  
dtypes: float64(3), int64(5), uint8(4)
memory usage: 7.0 MB


In [6]:
int_cols = ['gender','hypertension','heart_disease','diabetes','smoking_history_current','smoking_history_no info','smoking_history_non-smoker','smoking_history_past_smoker']

for col in int_cols:
    data[col] = data[col].astype('category')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96128 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   gender                       96128 non-null  category
 1   age                          96128 non-null  float64 
 2   hypertension                 96128 non-null  category
 3   heart_disease                96128 non-null  category
 4   bmi                          96128 non-null  float64 
 5   HbA1c_level                  96128 non-null  float64 
 6   blood_glucose_level          96128 non-null  int64   
 7   diabetes                     96128 non-null  category
 8   smoking_history_current      96128 non-null  category
 9   smoking_history_no info      96128 non-null  category
 10  smoking_history_non-smoker   96128 non-null  category
 11  smoking_history_past_smoker  96128 non-null  category
dtypes: category(8), float64(3), int64(1)
memory usage: 4.4 MB


## split the dataset

In [7]:
x = data.drop(['diabetes',"smoking_history_no info"],axis=1)
y = data["diabetes"]
x.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,smoking_history_current,smoking_history_non-smoker,smoking_history_past_smoker
0,0,80.00,0,1,25.19,6.60,140,0,1,0
1,0,54.00,0,0,27.32,6.60,80,0,0,0
2,1,28.00,0,0,27.32,5.70,158,0,1,0
3,0,36.00,0,0,23.45,5.00,155,1,0,0
4,1,76.00,1,1,20.14,4.80,155,1,0,0


In [8]:
train_feature, test_feature, train_label, test_label = train_test_split(x, y, test_size=0.2, random_state=42)
train_feature.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,smoking_history_current,smoking_history_non-smoker,smoking_history_past_smoker
39756,0,8.00,0,0,15.90,3.50,90,0,0,0
75304,1,65.00,1,1,28.49,8.80,145,0,1,0
19765,1,64.00,1,0,33.69,4.80,80,0,0,0
97291,1,49.00,0,0,21.84,5.00,130,0,1,0
37660,0,26.00,0,0,21.95,5.70,158,0,1,0


In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder

# Define your categorical and numeric columns
categorical_cols = ['gender', 'hypertension', 'heart_disease', 'smoking_history_current', 'smoking_history_non-smoker', 'smoking_history_past_smoker']
numeric_cols = ['age', 'blood_glucose_level','HbA1c_level', 'bmi']

# ... (load your train and test data)

models = {
    'Multiple Logistic': LogisticRegression(solver='liblinear'),
    'Ridge': LogisticRegression(penalty='l2', C=1.0),
    'Lasso': LogisticRegression(penalty='l1', C=1, solver='liblinear'),
    'Elastic-net':LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1, solver='saga'),
    'Random Forest': RandomForestClassifier(),
    'GausianNB': GaussianNB(),
    'XGBoost': XGBClassifier()
}

# Create an OneHotEncoder instance
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit and transform the encoder on your categorical columns
train_cat_encoded = encoder.fit_transform(train_feature[categorical_cols])
test_cat_encoded = encoder.transform(test_feature[categorical_cols])

# Concatenate the encoded categorical features with the numeric features
train_encoded = np.concatenate((train_cat_encoded, train_feature[numeric_cols]), axis=1)
test_encoded = np.concatenate((test_cat_encoded, test_feature[numeric_cols]), axis=1)

for name, model in models.items():
    print(name)
    print('=' * len(name))

    # Fit the model on the encoded data
    model.fit(train_encoded, train_label)
    
     # Training set
    train_pred = model.predict(train_encoded)
    train_acc = accuracy_score(train_label, train_pred)
    train_err = 1 - train_acc
    train_f1 = f1_score(train_label, train_pred, average='weighted')
    train_cm = confusion_matrix(train_label, train_pred)
    train_sensitivity = train_cm[1, 1] / (train_cm[1, 1] + train_cm[1, 0])
    train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])

    
    # Test set
    test_pred = model.predict(test_encoded)
    test_acc = accuracy_score(test_label, test_pred)
    test_err = 1 - test_acc
    test_f1 = f1_score(test_label, test_pred, average='weighted')
    test_cm = confusion_matrix(test_label, test_pred)
    test_sensitivity = test_cm[1, 1] / (test_cm[1, 1] + test_cm[1, 0])
    test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])


    print(f'Training Accuracy: {train_acc:.4f}')
    print(f'Training Error Rate: {train_err:.4f}')
    print(f'Training F1 Score: {train_f1:.4f}')
    print(f'Training Confusion Matrix:\n{train_cm}')
    print("Training sensitivity: ", train_sensitivity)
    print("Training specificity: ", train_specificity)
    print('\n')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test Error Rate: {test_err:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Confusion Matrix:\n{test_cm}')
    print("Test sensitivity: ", test_sensitivity)
    print("Test specificity: ", test_specificity)

    print('\n\n')
    

Multiple Logistic
Training Accuracy: 0.9593
Training Error Rate: 0.0407
Training F1 Score: 0.9564
Training Confusion Matrix:
[[69487   634]
 [ 2493  4288]]
Training sensitivity:  0.6323551098658015
Training specificity:  0.9909584860455498


Test Accuracy: 0.9573
Test Error Rate: 0.0427
Test F1 Score: 0.9541
Test Confusion Matrix:
[[17355   170]
 [  650  1051]]
Test sensitivity:  0.6178718400940623
Test specificity:  0.9902995720399429



Ridge
=====
Training Accuracy: 0.9528
Training Error Rate: 0.0472
Training F1 Score: 0.9487
Training Confusion Matrix:
[[69365   756]
 [ 2877  3904]]
Training sensitivity:  0.5757262940569238
Training specificity:  0.989218636357154


Test Accuracy: 0.9511
Test Error Rate: 0.0489
Test F1 Score: 0.9467
Test Confusion Matrix:
[[17329   196]
 [  745   956]]
Test sensitivity:  0.5620223398001176
Test specificity:  0.9888159771754637



Lasso
=====
Training Accuracy: 0.9593
Training Error Rate: 0.0407
Training F1 Score: 0.9563
Training Confusion Matrix:
[[

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

# Define your categorical and numeric columns
categorical_cols = ['gender', 'hypertension', 'heart_disease', 'smoking_history_current', 'smoking_history_non-smoker', 'smoking_history_past_smoker']
numeric_cols = ['age', 'blood_glucose_level','HbA1c_level', 'bmi']

# ... (load your train and test data)

models = {
    'Multiple Logistic': LogisticRegression(solver='liblinear'),
    'Ridge': LogisticRegression(penalty='l2', C=1.0),
    'Lasso': LogisticRegression(penalty='l1', C=1, solver='liblinear'),
    'Elastic-net':LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1, solver='saga'),
    'Random Forest': RandomForestClassifier(),
    'GausianNB': GaussianNB(),
    'XGBoost': XGBClassifier()
}

# Create an OneHotEncoder instance
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit and transform the encoder on your categorical columns
train_cat_encoded = encoder.fit_transform(train_feature[categorical_cols])
test_cat_encoded = encoder.transform(test_feature[categorical_cols])

# Concatenate the encoded categorical features with the numeric features
train_encoded = np.concatenate((train_cat_encoded, train_feature[numeric_cols]), axis=1)
test_encoded = np.concatenate((test_cat_encoded, test_feature[numeric_cols]), axis=1)

for name, model in models.items():
    print(name)
    print('=' * len(name))

    if name == 'Random Forest':
        param_grid = {
            'min_samples_split': [2, 5, 10],
            'max_depth': [None, 10, 20],
            'n_estimators': [50, 100, 200]
        }
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
        grid_search.fit(train_encoded, train_label)
        best_model = grid_search.best_estimator_
        model = best_model

    # Fit the model on the encoded data
    model.fit(train_encoded, train_label)
    
     # Training set
    train_pred = model.predict(train_encoded)
    train_acc = accuracy_score(train_label, train_pred)
    train_err = 1 - train_acc
    train_f1 = f1_score(train_label, train_pred, average='weighted')
    train_cm = confusion_matrix(train_label, train_pred)
    train_sensitivity = train_cm[1, 1] / (train_cm[1, 1] + train_cm[1, 0])
    train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])

    # Test set
    test_pred = model.predict(test_encoded)
    test_acc = accuracy_score(test_label, test_pred)
    test_err = 1 - test_acc
    test_f1 = f1_score(test_label, test_pred, average='weighted')
    test_cm = confusion_matrix(test_label, test_pred)
    test_sensitivity = test_cm[1, 1] / (test_cm[1, 1] + test_cm[1, 0])
    test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])

    print(f'Training Accuracy: {train_acc:.4f}')
    print(f'Training Error Rate: {train_err:.4f}')
    print(f'Training F1 Score: {train_f1:.4f}')
    print(f'Training Confusion Matrix:\n{train_cm}')
    print("Training sensitivity: ", train_sensitivity)
    print("Training specificity: ", train_specificity)
    print('\n')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test Error Rate: {test_err:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Confusion Matrix:\n{test_cm}')
    print("Test sensitivity: ", test_sensitivity)
    print("Test specificity: ", test_specificity)

    print('\n\n')


Multiple Logistic
Training Accuracy: 0.9593
Training Error Rate: 0.0407
Training F1 Score: 0.9564
Training Confusion Matrix:
[[69487   634]
 [ 2493  4288]]
Training sensitivity:  0.6323551098658015
Training specificity:  0.9909584860455498


Test Accuracy: 0.9573
Test Error Rate: 0.0427
Test F1 Score: 0.9541
Test Confusion Matrix:
[[17355   170]
 [  650  1051]]
Test sensitivity:  0.6178718400940623
Test specificity:  0.9902995720399429



Ridge
=====
Training Accuracy: 0.9528
Training Error Rate: 0.0472
Training F1 Score: 0.9487
Training Confusion Matrix:
[[69365   756]
 [ 2877  3904]]
Training sensitivity:  0.5757262940569238
Training specificity:  0.989218636357154


Test Accuracy: 0.9511
Test Error Rate: 0.0489
Test F1 Score: 0.9467
Test Confusion Matrix:
[[17329   196]
 [  745   956]]
Test sensitivity:  0.5620223398001176
Test specificity:  0.9888159771754637



Lasso
=====
Training Accuracy: 0.9593
Training Error Rate: 0.0407
Training F1 Score: 0.9564
Training Confusion Matrix:
[[

In [11]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 100}


# ensemble technique - stacking

1. Stacking: It is an ensemble method that combines multiple models (classification or regression) via meta-model (meta-classifier or meta-regression). The base models are trained on the complete dataset, then the meta-model is trained on features returned (as output) from base models. The base models in stacking are typically different. The meta-model helps to find the features from base models to achieve the best accuracy.
    
    Stacking is a bit different from the basic ensembling methods because it has first-level and second-level models. Stacking features are first extracted by training the dataset with all the first-level models. A first-level model is then using the train stacking features to train the model than this model predicts the final output with test stacking features.

Algorithm:

        Split the train dataset into n parts
        A base model (say linear regression) is fitted on n-1 parts and predictions are made for the nth part. This is done for each one of the n part of the train set.
        The base model is then fitted on the whole train dataset.
        This model is used to predict the test dataset.
        The Steps 2 to 4 are repeated for another base model which results in another set of predictions for the train and test dataset.
        The predictions on train data set are used as a feature to build the new model.
        This final model is used to make the predictions on test dataset
        
https://www.geeksforgeeks.org/ensemble-methods-in-python/

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from vecstack import stacking


# Train XGBoost and Random Forest models
xgb_model = XGBClassifier(learning_rate= 0.1, max_depth = 3, n_estimators = 200)
rf_model = RandomForestClassifier()
lr_model = LogisticRegression(solver='liblinear')

# Create a stacking model with a meta-model (Logistic Regression)
models = [xgb_model, rf_model, lr_model]
meta_model = LogisticRegression(solver='liblinear')

# Perform stacking to generate ensemble predictions
S_train, S_test = stacking(models, train_encoded, train_label, test_encoded,
                           regression=False, metric=accuracy_score,
                           n_folds=5, shuffle=True, random_state=42,
                           verbose=2)

# Train the meta-model on the stacked predictions
meta_model.fit(S_train, train_label)

# Ensemble predictions using the meta-model
ensemble_train_preds = meta_model.predict(S_train)
ensemble_test_preds = meta_model.predict(S_test)

# Calculate accuracy
ensemble_train_acc = accuracy_score(train_label, ensemble_train_preds)
ensemble_test_acc = accuracy_score(test_label, ensemble_test_preds)

print("Ensemble Train Accuracy:", ensemble_train_acc)
print("Ensemble Test Accuracy:", ensemble_test_acc)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBClassifier]
    fold  0:  [0.97249854]
    fold  1:  [0.97165334]
    fold  2:  [0.96911573]
    fold  3:  [0.97243173]
    fold  4:  [0.97204161]
    ----
    MEAN:     [0.97154819] + [0.00125341]
    FULL:     [0.97154820]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96957285]
    fold  1:  [0.96879267]
    fold  2:  [0.96657997]
    fold  3:  [0.97041612]
    fold  4:  [0.96905072]
    ----
    MEAN:     [0.96888247] + [0.00127801]
    FULL:     [0.96888247]

model  2:     [LogisticRegression]
    fold  0:  [0.96053573]
    fold  1:  [0.96001560]
    fold  2:  [0.95715215]
    fold  3:  [0.95864759]
    fold  4:  [0.95994798]
    ----
    MEAN:     [0.95925981] + [0.00122391]
    FULL:     [0.95925984]

Ensemble Train Accuracy: 0.9712361186965228
Ensemble Test Accuracy: 0.9691563507749922


In [13]:
import numpy as np
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from vecstack import stacking


# Train XGBoost and Random Forest models
xgb_model = XGBClassifier()
rf_model = RandomForestClassifier()
lr_model = LogisticRegression()

# Create a stacking model with a meta-model (Logistic Regression)
models = [xgb_model, rf_model, lr_model]
meta_model = LogisticRegression()

# Perform stacking to generate ensemble predictions
S_train, S_test = stacking(models, train_encoded, train_label, test_encoded,
                           regression=False, metric=accuracy_score,
                           n_folds=5, shuffle=True, random_state=42,
                           verbose=2)

# Train the meta-model on the stacked predictions
meta_model.fit(S_train, train_label)

# Ensemble predictions using the meta-model
ensemble_train_preds = meta_model.predict(S_train)
ensemble_test_preds = meta_model.predict(S_test)

# Calculate accuracy
ensemble_train_acc = accuracy_score(train_label, ensemble_train_preds)
ensemble_test_acc = accuracy_score(test_label, ensemble_test_preds)

print("Ensemble Train Accuracy:", ensemble_train_acc)
print("Ensemble Test Accuracy:", ensemble_test_acc)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBClassifier]
    fold  0:  [0.97048306]
    fold  1:  [0.97048306]
    fold  2:  [0.96827048]
    fold  3:  [0.97145644]
    fold  4:  [0.97015605]
    ----
    MEAN:     [0.97016982] + [0.00104492]
    FULL:     [0.97016983]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96957285]
    fold  1:  [0.96762239]
    fold  2:  [0.96618986]
    fold  3:  [0.97080624]
    fold  4:  [0.96846554]
    ----
    MEAN:     [0.96853138] + [0.00158575]
    FULL:     [0.96853138]

model  2:     [LogisticRegression]
    fold  0:  [0.95351408]
    fold  1:  [0.95364411]
    fold  2:  [0.95539662]
    fold  3:  [0.95682705]
    fold  4:  [0.95942783]
    ----
    MEAN:     [0.95576194] + [0.00220203]
    FULL:     [0.95576188]

Ensemble Train Accuracy: 0.9704559049179475
Ensemble Test Accuracy: 0.9685842088838031


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from vecstack import stacking


# Train XGBoost and Random Forest models
xgb_model = XGBClassifier()
rf_model = RandomForestClassifier()
llr_model = LogisticRegression(penalty='l1', C=1, solver='liblinear')

# Create a stacking model with a meta-model (Logistic Regression)
models = [xgb_model, rf_model, llr_model]
meta_model = LogisticRegression(penalty='l1', C=1, solver='liblinear')

# Perform stacking to generate ensemble predictions
S_train, S_test = stacking(models, train_encoded, train_label, test_encoded,
                           regression=False, metric=accuracy_score,
                           n_folds=5, shuffle=True, random_state=42,
                           verbose=2)

# Train the meta-model on the stacked predictions
meta_model.fit(S_train, train_label)

# Ensemble predictions using the meta-model
ensemble_train_preds = meta_model.predict(S_train)
ensemble_test_preds = meta_model.predict(S_test)

# Calculate accuracy
ensemble_train_acc = accuracy_score(train_label, ensemble_train_preds)
ensemble_test_acc = accuracy_score(test_label, ensemble_test_preds)

print("Ensemble Train Accuracy:", ensemble_train_acc)
print("Ensemble Test Accuracy:", ensemble_test_acc)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBClassifier]
    fold  0:  [0.97048306]
    fold  1:  [0.97048306]
    fold  2:  [0.96827048]
    fold  3:  [0.97145644]
    fold  4:  [0.97015605]
    ----
    MEAN:     [0.97016982] + [0.00104492]
    FULL:     [0.97016983]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96957285]
    fold  1:  [0.96827254]
    fold  2:  [0.96579974]
    fold  3:  [0.97041612]
    fold  4:  [0.96963589]
    ----
    MEAN:     [0.96873943] + [0.00162292]
    FULL:     [0.96873943]

model  2:     [LogisticRegression]
    fold  0:  [0.96060074]
    fold  1:  [0.96008062]
    fold  2:  [0.95747724]
    fold  3:  [0.95845254]
    fold  4:  [0.95988296]
    ----
    MEAN:     [0.95929882] + [0.00115601]
    FULL:     [0.95929885]

Ensemble Train Accuracy: 0.9707419833034251
Ensemble Test Accuracy: 0.9688442733797982


In [15]:
0.9712231151335466-0.9691043378757932 #xgb with hyperpara

0.002118777257753446

In [16]:
0.9712751293854516- 0.9691563507749922 ##log(solver='liblinear') + #xgb with hyperpara

0.002118778610459393

In [17]:
0.9705859405477101-0.9685321959846042 #none

0.0020537445631059192

In [18]:
0.9706379547996151-0.9687402475814002 #log(solver='liblinear')

0.0018977072182149168